In [1]:
dataName = 'adani'
dataFrequency = '1h'
featureIndexStamp = "013_"
variation_degree=20
requiredMinimumCorrelation = .2
autoConfigFileRelativePath = '\\src\config\\autoConfig\\config.json'
KEY_preProcessedDataFilePath='preProcessedDataFilePath'

print('done')

done


In [2]:
# Include python files defined in the library folder to help facilitate the stock price Prediction 
import os
import sys  
module_path = os.path.abspath('../../'+os.path.join('.'))

sys.path.append(module_path)
print('done')

from dataPreparation import *
from dataPreprocessing import *
from utilities import *

from dataPreparation.featurePreparation import doBasicOperation
#from dataPreparation.featurePreparation import doFeatureAssessment

_basicDf,_variation_degree,_preProcessedDataFilePath,_autoConfigData,_configFilePath = doBasicOperation(dataName,
       dataFrequency,autoConfigFileRelativePath,KEY_preProcessedDataFilePath,variation_degree)


done
into method doBasicOperation
jupyterNodePath >>> C:\Users\kalagi01\Desktop\dev\eraiVersion2
configFilePath >>> C:\Users\kalagi01\Desktop\dev\eraiVersion2\src\config\autoConfig\config.json
added INPUT FEATURES >>> 4 count >>> open-high-low-close
before return statement of method doBasicOperation 


In [3]:
#temporary stuff
import traceback

traceback_template = '''Traceback (most recent call last):
  File "%(filename)s", line %(lineno)s, in %(name)s
%(type)s: %(message)s\n''' # Skipping the "actual line" item

#from dataPreparation.featurePreparation import createNewTrainingSetWithFeatureVariations

In [4]:
def suffixColumnsWithLabel(df,label):
    import pandas as pd
    ref=pd.concat([df],axis=1)
    ref.columns+=label
    return ref


def doFeatureAssessment(newFeatureDf,basicDf,variation_degree,preProcessedDataFilePath,autoConfigData,
    configFilePath,requiredMinimumCorrelation,featureIndexStamp,dataName,dataFrequency,useVersion2=False):
    import numpy as np
    import pandas as pd

    from utilities.fileFolderManipulations import getJupyterRootDirectory
    from utilities.fileFolderManipulations import getParentFolder
    from utilities.fileFolderManipulations import createFolder
    from utilities.fileFolderManipulations import deleteFile
    
    from dataPreprocessing.basicRawDataProcess import setAutoConfigData
    
    try:

        featureOfInterest = newFeatureDf.columns
        if variation_degree==0:
            newTrainingSetDf=pd.concat([basicDf,newFeatureDf],axis=1)
        else:
            if useVersion2 :
                newTrainingSetDf = createNewTrainingSetWithFeatureVariations_v2(basicDf,newFeatureDf,featureOfInterest,variation_degree) 
            else :
                newTrainingSetDf = createNewTrainingSetWithFeatureVariations(basicDf,newFeatureDf,featureOfInterest,variation_degree) 

        return newTrainingSetDf
        correlation = newTrainingSetDf.corr()

        reasonableCorelation = correlation.loc[ (np.abs(correlation['open'])>requiredMinimumCorrelation) & 
        (np.abs(correlation['high'])>requiredMinimumCorrelation) &
        (np.abs(correlation['low'])>requiredMinimumCorrelation) & 
        (np.abs(correlation['close'])>requiredMinimumCorrelation)]

        # create necessary file folder structure for storing and filtering features
        preprocessedFolderPath = getParentFolder(preProcessedDataFilePath)
        outputFolderPath = getParentFolder(preprocessedFolderPath)

        featuresFolder = outputFolderPath+"\\features"
        createFolder(featuresFolder)

        rawFeaturesFolder = featuresFolder+"\\rawFeatures"
        createFolder(rawFeaturesFolder)

        filteredFeaturesFolder = featuresFolder+"\\filteredFeatures"
        createFolder(filteredFeaturesFolder)

        correlationsFolder = featuresFolder+"\\correlations"
        createFolder(correlationsFolder)

        reasonableCorrelationsFolder = correlationsFolder+"\\reasonableCorrelations"
        createFolder(reasonableCorrelationsFolder)

        trainableFeaturesListtFilePath = filteredFeaturesFolder+"\\"+featureIndexStamp+featureOfInterest+"_trainableFeaturesList.csv"
        currentFeatureListFilePath = rawFeaturesFolder+"\\"+featureIndexStamp+featureOfInterest+"_variations_list.csv"
        currentFeatureCorrelationListFilePath = correlationsFolder+"\\"+featureIndexStamp+featureOfInterest+"_variations_correlation_list.csv"
        reasonableCorelationListFilePath = reasonableCorrelationsFolder+"\\"+featureIndexStamp+featureOfInterest+"_variations_reasonable_correlation_list.csv"

        deleteFile(trainableFeaturesListtFilePath)
        deleteFile(currentFeatureListFilePath)
        deleteFile(currentFeatureCorrelationListFilePath)
        deleteFile(reasonableCorelationListFilePath)

        # store output information related to current 
        newTrainingSetDf.to_csv(currentFeatureListFilePath, sep=',', index=False)
        correlation.to_csv(currentFeatureCorrelationListFilePath, sep=',', index=True)
        reasonableCorelation.to_csv(reasonableCorelationListFilePath, sep=',', index=True)

        if len(reasonableCorelation.index)>4:    
            # store trainable features in global file - to be used by other training feature creation procedures    
            newFilteredTrainableFeaturesDf = newTrainingSetDf[[filteredIndex for filteredIndex in reasonableCorelation.index] ]
            trainableFeaturesDf=newFilteredTrainableFeaturesDf.drop(columns=["open","close","high","low"])    
            # trainableFeaturesDf = getTrainableFeaturesListDf(trainableFeaturesListtFilePath)
            # if trainableFeaturesDf is None:
            #     trainableFeaturesDf= newFilteredTrainableFeaturesDf
            # else:        
            #     # newFilteredTrainableFeaturesDf=newFilteredTrainableFeaturesDf.drop(columns=["open","close","high","low"])    
            #     # trainableFeaturesDf = pd.concat([trainableFeaturesDf,newFilteredTrainableFeaturesDf],axis=1)
            #     for index in reasonableCorelation:
            #         try:
            #             trainableFeaturesDf[index] = newFilteredTrainableFeaturesDf[index]
            #         except KeyError:
            #             print ('key error >>>' + index)
            
            if not trainableFeaturesDf is None or trainableFeaturesDf.shape[1]>0:
                trainableFeaturesDf.to_csv(trainableFeaturesListtFilePath, sep=',', index=False)

            # assertions
            print("newTrainingSetDf shape>>>"+str(newTrainingSetDf.shape[0])+","+str(newTrainingSetDf.shape[1]))
            print("trainableFeaturesDf shape>>>"+str(trainableFeaturesDf.shape[0])+","+str(trainableFeaturesDf.shape[1]))
            
            autoConfigData[dataName][dataFrequency].update({'trainableFeaturesListtFile':trainableFeaturesListtFilePath})
            setAutoConfigData(configFilePath,autoConfigData)
        else:
            trainableFeaturesDf = getTrainableFeaturesListDf(trainableFeaturesListtFilePath)

        return correlation, reasonableCorelation ,newTrainingSetDf ,trainableFeaturesDf
    except:
        print("Error executing method >>> ")
        # exc_type, exc_obj, exc_tb = sys.exc_info()
        # fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        # print("Unexpected error:", sys.exc_info())
        # print(exc_type, fname, exc_tb.tb_lineno)
        
        # http://docs.python.org/2/library/sys.html#sys.exc_info
        exc_type, exc_value, exc_traceback = sys.exc_info() # most recent (if any) by default
        
        '''
        Reason this _can_ be bad: If an (unhandled) exception happens AFTER this,
        or if we do not delete the labels on (not much) older versions of Py, the
        reference we created can linger.

        traceback.format_exc/print_exc do this very thing, BUT note this creates a
        temp scope within the function.
        '''

        traceback_details = {
                            'filename': exc_traceback.tb_frame.f_code.co_filename,
                            'lineno'  : exc_traceback.tb_lineno,
                            'name'    : exc_traceback.tb_frame.f_code.co_name,
                            'type'    : exc_type.__name__,
                            'message' : traceback.extract_tb(exc_traceback)
                            }
        
        del(exc_type, exc_value, exc_traceback) # So we don't leave our local labels/objects dangling
        # This still isn't "completely safe", though!
        # "Best (recommended) practice: replace all exc_type, exc_value, exc_traceback
        # with sys.exc_info()[0], sys.exc_info()[1], sys.exc_info()[2]
        
        print
        print(traceback.format_exc())
        print
        print(traceback_template % traceback_details)
        print

        #traceback.print_exception()
        raise



In [80]:
def createNewTrainingSetWithFeatureVariations_v2(basicDf,newFeatureDf,featureOfInterest,variation_degree) :
    import pandas as pd
    import numpy as np
    from tqdm import tqdm

    try:
        # Create and register a new `tqdm` instance with `pandas`
        # (can use tqdm_gui, optional kwargs, etc.)
        tqdm.pandas()


        featureVariants=[[
                            np.exp(suffixColumnsWithLabel(newFeatureDf,'_exp_'+str(iterator))*iterator), 
                            np.exp(suffixColumnsWithLabel(newFeatureDf,'_exp_inv_'+str(iterator))*iterator*-1),
                            np.power(suffixColumnsWithLabel(newFeatureDf,'_pow_'+str(iterator)),iterator),
                            np.power(suffixColumnsWithLabel(newFeatureDf,'_pow_inv_'+str(iterator)),iterator*-1)
        ] for iterator in range(1,variation_degree+1)]
        
        
        row=len(featureVariants[0][0])
        col=len(featureVariants[0])
        featureVariantCummList=[print(str(i)+","+str(j)) for i in range(0,row-1) for j in range(0,col-1) if i>8300]
        #featureVariantCummList=[featureVariants[i][j] for i in range(0,row-1) for j in range(0,col-1)]
        
        
        return featureVariantCummList
        newVariantsDf=pd.concat(featureVariantCummList,axis=1)
        newTrainingSetDf= pd.concat([basicDf,newVariantsDf,newFeatureDf],axis=1)
        #newTrainingSetDf= pd.concat([basicDf,newFeatureDf,newVariantsDf],axis=1)
        
        
          
        
#         for iterator in range(1,variation_degree_count+1):
#             row[featureName+'_exp_1'] = np.exp(featureVal)
#             row[featureName+'_exp_inv_1'] = np.exp(-1*featureVal)
        
#             if iterator>1:
#                 
#                 if val>0:
#                     row[featureName+'_log_times_'+str(iterator)] = np.log(val*iterator)
#                 elif val<0:
#                     row[featureName+'_log_times_'+str(iterator)] = -np.log(-1*val*iterator)
            
#         return row 

        
    
        return newTrainingSetDf
    except:
        print("Error executing method >>> ")
        # exc_type, exc_obj, exc_tb = sys.exc_info()
        # fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        # print("Unexpected error:", sys.exc_info())
        # print(exc_type, fname, exc_tb.tb_lineno)
        
        # http://docs.python.org/2/library/sys.html#sys.exc_info
        exc_type, exc_value, exc_traceback = sys.exc_info() # most recent (if any) by default
        
        '''
        Reason this _can_ be bad: If an (unhandled) exception happens AFTER this,
        or if we do not delete the labels on (not much) older versions of Py, the
        reference we created can linger.

        traceback.format_exc/print_exc do this very thing, BUT note this creates a
        temp scope within the function.
        '''

        traceback_details = {
                            'filename': exc_traceback.tb_frame.f_code.co_filename,
                            'lineno'  : exc_traceback.tb_lineno,
                            'name'    : exc_traceback.tb_frame.f_code.co_name,
                            'type'    : exc_type.__name__,
                            'message' : traceback.extract_tb(exc_traceback)
                            }
        
        del(exc_type, exc_value, exc_traceback) # So we don't leave our local labels/objects dangling
        # This still isn't "completely safe", though!
        # "Best (recommended) practice: replace all exc_type, exc_value, exc_traceback
        # with sys.exc_info()[0], sys.exc_info()[1], sys.exc_info()[2]
        
        print
        print(traceback.format_exc())
        print
        print(traceback_template % traceback_details)
        print

        #traceback.print_exception()
        raise



In [81]:
import pandas as pd
import numpy as np

featureOfInterest = 'prior_days'
_basicDf=_basicDf[1:10]
#newFeatureDf = ((_basicDf['open']+_basicDf['high'])/2).rename(featureOfInterest)
inputRawProcessedDataDF = pd.read_csv(_preProcessedDataFilePath) 

# newFeatureDf = np.divide(np.sqrt(np.sum(np.square(inputRawProcessedDataDF[['open','close','high','low']]),axis=1)),4)
# newFeatureDf = newFeatureDf.rename(featureOfInterest)
import pandas as pd
import numpy as np

featureName=['prior_holidays','scarcity_by_prior_holidays','magnitudeTimesScarcity_by_prior_holidays']

priorHolidaysDataDf = inputRawProcessedDataDF[['open','close','high','low','prior_holidays']]
m = priorHolidaysDataDf.shape[0]

prior_holidays_unique_values = priorHolidaysDataDf[featureName[0]].unique()

dataByScarcityOfPriorHolidays=priorHolidaysDataDf[featureName[0]].rename(featureName[1])

priorHolidaysDataDf=pd.concat([priorHolidaysDataDf,dataByScarcityOfPriorHolidays],axis=1)

for unq_val in prior_holidays_unique_values:
    matches=priorHolidaysDataDf.loc[priorHolidaysDataDf[featureName[0]]==unq_val]    
    priorHolidaysDataDf[featureName[0]][priorHolidaysDataDf[featureName[1]]==unq_val]=100*(m-matches.shape[0])/m
  
magnitudeDf= np.divide(np.sqrt(np.sum(np.square(priorHolidaysDataDf[['open','close','high','low']]),axis=1)),4)

magnitudeTimesScarcityDf = np.multiply(magnitudeDf,dataByScarcityOfPriorHolidays)

priorHolidaysDataDf=pd.concat([priorHolidaysDataDf,magnitudeTimesScarcityDf.rename(featureName[2])],axis=1)            

newFeatureDf=priorHolidaysDataDf.drop({'open','close','high','low'},axis=1)
#priorHolidaysDataDf.corr()

_basicDf=inputRawProcessedDataDF[['open','close','high','low']]

# _correlation, _reasonableCorelation ,_newTrainingSetDf ,_trainableFeaturesDf = doFeatureAssessment(newFeatureDf,
#             _basicDf,_variation_degree,_preProcessedDataFilePath,_autoConfigData,
#             _configFilePath,requiredMinimumCorrelation,featureIndexStamp,
#             dataName,dataFrequency,useVersion2=True)
_trainableFeaturesDf = doFeatureAssessment(newFeatureDf,
            _basicDf,_variation_degree,_preProcessedDataFilePath,_autoConfigData,
            _configFilePath,requiredMinimumCorrelation,featureIndexStamp,
            dataName,dataFrequency,useVersion2=True)
_trainableFeaturesDf

#newFeatureDf.columns
# print('done')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in power


8301,0
8301,1
8301,2
8302,0
8302,1
8302,2
8303,0
8303,1
8303,2

In [75]:
len(_trainableFeaturesDf[0])
len(_trainableFeaturesDf[0][0])

8350

In [9]:
len

<function len(obj, /)>

In [10]:
newFeatureDf=pd.DataFrame({"a":[-1,0,3],'b':[-1,2,3],'c':[1,0,-3]})
newFeatureDf.head()

,a,b,c
0,-1,-1,1
1,0,2,0
2,3,3,-3


In [11]:
#np.exp(df,[ iterator for iterator in range(variation_degree+1)])
#pd.concat([np.exp((df.rename(columns={"a":"a_sq"}))*2),np.exp(df*3)],axis=1)

# mdf=df
# for logItr in range(21)
# mdf[mdf<0]=-np.log(-1*mdf[mdf<0])
# mdf[mdf>0]=np.log(mdf[mdf>0])
# #np.log(2*df)
# df


In [12]:
df.columns+="_test"
df

NameError: name 'df' is not defined

In [ ]:
x=[]
x.append(1)
x